In [ ]:
import pandas as pd
import datetime
import requests
import urllib.parse
import requests

apiKey = '218c777c-8a2b-4c3f-bc1a-4b5160beec48'
secretKey = 'rt7sr4wtzp'
redirectUrl = 'https://127.0.0.1:5000/'
rurl = urllib.parse.quote(redirectUrl,safe="")
rurl

In [ ]:
url = f'https://api-v2.upstox.com/login/authorization/dialog?response_type=code&client_id={apiKey}&redirect_uri={rurl}'
print(url)

In [ ]:

url = 'https://api-v2.upstox.com/login/authorization/token'
code = '_35LB'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'code': code,
    'client_id': apiKey,
    'client_secret': secretKey,
    'redirect_uri': redirectUrl,
    'grant_type': 'authorization_code'
}

response = requests.post(url, headers=headers, data=data)
json_response = response.json()

json_response

Access Token Generation

In [ ]:
access_token = json_response['access_token']
access_token

Data Reading

In [ ]:
df = pd.read_csv("https://assets.upstox.com/market-quote/instruments/exchange/complete.csv.gz")
df .columns
df['exchange'].unique()
df[df['exchange'] == 'NSE_FO']
#df[df['exchange'] == 'MCX_FO']

Extracting the required insturments existing contracts  of the particular excange with the lot size

In [ ]:
df[(df['exchange'] == 'NSE_FO') & (df['instrument_type'] == 'OPTIDX') & (df['lot_size']==25)]
#df[(df['exchange'] == 'MCX_FO')  & (df['lot_size']==100)] //to switch the exchange 


Selecting the required Instrument with lot size and expiry Date .

In [ ]:
df[(df['exchange'] == 'NSE_FO') & (df['instrument_type'] == 'OPTIDX') & (df['lot_size']==25)&(df['expiry']=='2024-09-19')]

Extracted all the strike which is active  or you can reteive with premiums providing range.

In [ ]:
import pandas as pd

# Your input dataframe
input_df = df[(df['exchange'] == 'NSE_FO') & 
              (df['instrument_type'] == 'OPTIDX') & 
              (df['lot_size'] == 25) & 
              (df['expiry'] == '2024-09-19')]

# # Selecting the middle 50 up and 50 down trading symbols
middle_index = len(input_df) // 1 
selected_df = input_df.iloc[middle_index - 50: middle_index + 50]

# Selecting only required columns
selected_df = selected_df[['instrument_key', 'tradingsymbol','last_price' ,'expiry', 'strike', 'exchange_token']]

# Filtering based on last_price range
selected_df_filtered = selected_df[(selected_df['last_price'] >= 0.5) & (selected_df['last_price'] <= 1000)]

# Print the resulting dataframe
print(selected_df_filtered)
selected_df_filtered


Extracting 1 day Historical data of extracted Contracts

In [ ]:
#1d OHLC

# Define the URL for the OHLC API endpoint
url = 'https://api.upstox.com/v2/market-quote/ohlc'

# Define a list to store OHLC data for each instrument key
ohlc_data_list = []

# Extract instrument keys from the selected_df DataFrame
instrument_keys = selected_df_filtered['instrument_key'].tolist()

# Set up headers with access token
headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
}

# Define the interval for OHLC data (e.g., 1d for daily OHLC data)
interval = '1d'

# Iterate over each instrument key and fetch OHLC data
for instrument_key in instrument_keys:
    # Define the query parameters
    params = {
        'instrument_key': instrument_key,
        'interval': interval
    }

    # Send a GET request to the OHLC endpoint for the current instrument key
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and append the OHLC data to the list
        ohlc_data_list.append(response.json())
    else:
        # Print an error message if the request was not successful
        print(f"Error for instrument key {instrument_key}: {response.status_code} - {response.text}")

# Print the OHLC data for all instrument keys
for ohlc_data in ohlc_data_list:
    print(ohlc_data)
    


 Now extracting 1 min Historical data of extracted Contracts or It can also extract 30minute

In [ ]:
#1min OHLC


import requests

# Define your access token
access_token = access_token

# Define the base URL for historical candle data
base_url = 'https://api.upstox.com/v2/historical-candle/'

# Define the interval for historical candle data (1 minute)
interval = '1minute'

# Define the start and end dates for the historical data (e.g., last 30 days)
end_date = '2024-09-12'  # Today's date
start_date = '2024-09-12'  # __ days ago from today

# Define a list to store 1-minute historical data for each instrument
historical_data_list = []

# Set up headers with access token
headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
}

# Iterate over each instrument in the selected_df DataFrame
for index, row in selected_df_filtered.iterrows():
    # Construct the URL for the current instrument
    instrument_key = row['instrument_key']
    url = f"{base_url}{instrument_key}/{interval}/{end_date}/{start_date}"

    # Send a GET request to fetch historical candle data for the current instrument
    response = requests.get(url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and append the historical data to the list
        historical_data_list.append(response.json())
    else:
        # Print an error message if the request was not successful
        print(f"Error for instrument key {instrument_key}: {response.status_code} - {response.text}")

# Print the historical data for all instruments
for historical_data in historical_data_list:
    print(historical_data)


To get the extracted in the Proper data frame to increase readability and saved to the local directory .

In [ ]:
import requests
import pandas as pd

def fetch_historical_candle_data(instrument_key, trading_symbol, interval, start_date, end_date, access_token):
    base_url = 'https://api.upstox.com/v2/historical-candle/'
    url = f"{base_url}{instrument_key}/{interval}/{end_date}/{start_date}"
    
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        historical_data = response.json()
        candles_data = historical_data['data']['candles']
        instrument_data = []

        for candle in candles_data:
            timestamp = candle[0].replace('+05:30', '')
            open_price = candle[1]
            high_price = candle[2]
            low_price = candle[3]
            close_price = candle[4]
            volume = candle[5]
            open_interest = candle[6]
            date = timestamp.split('T')[0]  # Extract date
            time = timestamp.split('T')[1]  # Extract time
            trading_symbol_parts = trading_symbol.split('249')  # Split at '242'[24 (Year) and 2 (month)] in 'BANKNIFTY2422143700PE'in case of monthly use 24M(march month)
            instrument_code = trading_symbol_parts[0]
            strike = trading_symbol_parts[1][2:]  # Extract the strike price (excluding '242')
            instrument_data.append([instrument_key, trading_symbol, instrument_code, strike, timestamp, date, time, open_price, high_price, low_price, close_price, volume, open_interest])

        return instrument_data
    else:
        print(f"Error for instrument key {instrument_key}: {response.status_code} - {response.text}")
        return None

def fetch_all_historical_candle_data(selected_df, interval, start_date, end_date, access_token):
    historical_data_list = []

    for index, row in selected_df.iterrows():
        instrument_key = row['instrument_key']
        trading_symbol = row['tradingsymbol']
        instrument_data = fetch_historical_candle_data(instrument_key, trading_symbol, interval, start_date, end_date, access_token)
        
        if instrument_data:
            historical_data_list.extend(instrument_data)

    return historical_data_list

def main():
    # Define your access token
    access_token = 'access_token'

    # Define the interval for historical candle data (1 minute)
    interval = '1minute'

    # Define the start and end dates for the historical data (e.g., last 30 days)
    end_date = '2024-09-12'  # Today's date
    start_date = '2024-09-12'  # ___ days ago from today

    # Fetch all historical candle data for selected instruments
    historical_data_list = fetch_all_historical_candle_data(selected_df_filtered, interval, start_date, end_date, access_token)

    # Create a DataFrame with headers
    column_names = ['Instrument Key', 'Trading Symbol', 'Instrument Code', 'Strike', 'Timestamp', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']
    df = pd.DataFrame(historical_data_list, columns=column_names)

    # Save the DataFrame to a CSV file
    csv_file_path = 'banknifty Sep 12 .csv'
    df.to_csv(csv_file_path, index=False)
    print(f"CSV file has been saved successfully at: {csv_file_path}")

if __name__ == "__main__":
    main()
